### How can we choose a proper relaxation time list in our PMMA contact problem?

Our viscoelastic solver is based on linear viscoelastic model, with widely-used generalized Maxwell model. The parameters, i.e. relaxation times and elastic moduli of generalized Maxwell model are determined from either a relaxation or a Dynamical Mechanical Analysis(DMA) experiments. In Jalocha(2015)[1], they propose two techniques to identify an optimal series of relaxation times. In the case of the relaxation experiment relaxation times will be optimized from the numerical integration of the measured relaxation spectrum. In the case of the DMA experiments we show that mathematical results obtained by Krein and Nudelmann can be used to determine the complete series of relaxation times.

The main constitutive unknown is the relaxation spectrum $H(\tau)$ (Findley et al., 1976) which is related to the relaxation modulus $E(t)$ by:

$$
E(t)=E_0+\int_{-\infty}^{\infty} H(\tau) e^{-\frac{t}{\tau}} d \ln (\tau)
$$


For practical reasons it is convenient to use a model, where the continuous spectrum of relaxation $H(\tau)$ is replaced with a finite spectrum $\hat{H}(\tau)$:

$$
\hat{H}(\tau)=\sum_{i=1}^n E_i \delta\left(1-\frac{\tau}{\tau_i}\right)
$$

In the discrete case of Prony series, the relaxation modulus $E(t)$ is represented as:

$$
E(t)=E_0+\sum_{i=1}^n E_i e^{-\frac{t}{\tau_i}}
$$

In the frequency domain, the dynamical modulus $E^*(\omega)$ becomes:

$$
E^{\prime}(\omega)=E_0+\sum_{i=1}^n \frac{E_i \omega^2 \tau_i^2}{1+\omega^2 \tau_i^2} \quad E^{\prime \prime}(\omega)=\sum_{i=1}^n \frac{E_i \omega \tau_i}{1+\omega^2 \tau_i^2}
$$

#### DMA experiment

For a given frequency $\omega$, the complex dynamical modulus is defined as:

$$
E^*(\omega)=\frac{\sigma^*}{\varepsilon^*}=E^{\prime}(\omega)+i E^{\prime \prime}(\omega)
$$



##### *HW method*

In Jalocha(2015)[1], they first reviewed *HW* method proposed by *Honerkamp (1989)* and *Honerkamp and Weese (1990)*, where they first choose arbitrary Maxwell branches and solve the dynamical modulus $E^{\prime}(\omega)$ and $E^{\prime \prime}(\omega)$ equation, $\mathbf{E}^{\prime}=\mathbf{E}_{\mathbf{0}}+\mathbf{A E}$ and $\mathbf{E}^{\prime \prime}=\mathbf{B E}$, in the frequency domain, where the matrix $\mathbf{A}$ and $\mathbf{B}$ are defined as:

$$
\mathbf{A}=\left(a_{i j}\right) \quad a_{j i}=\frac{\omega_j^2 \tau_i^2}{1+\omega_j^2 \tau_i^2}
$$

$$
\mathbf{B}=\left(b_{i j}\right) \quad b_{j i}=\frac{\omega_j \tau_i}{1+\omega_j^2 \tau_i^2}
$$



##### *Proposed procedure: the KN + HW method* in Jalocha(2015)[1]

The method proposed permits to replace the arbitrary choice of the relaxation times with an exact computation of the series of relaxation times. This novelty eliminates the first step of the *HW method*, where the relaxation times are generally imposed as a linear series on the logarithmic scale.(More detailed mathmatical deduce is in Appendix)

- Step 1: Build the two complex matrices $\mathbf{M}^1$ and $\mathbf{M}^2$ defined by their components:

$$
M_{k l}^1=i \frac{\overline{E_k^*}-E_l^*}{\omega_k+\omega_l} \quad M_{k l}^2=\frac{\frac{\overline{E_k^*}}{\omega_k}+\frac{E_l^*}{\omega_l}}{\omega_k+\omega_l}
$$

where $E_k^*=E_k^{\prime}+i E_k^{\prime \prime}$ with $k, l=1, m$ and $\overline{E_k^*}$ the conjugate complex number of $E_k^*$.

- Step 2: Check the positive definition of matrices $\mathbf{M}^1\left(E^*, \omega\right)$ and $\mathbf{M}^2\left(E^*, \omega\right)$.

- Step 3: Compute the $\mathbf{v}$ and $\mathbf{w}$ the eigenvectors spanning the kernel of $\mathbf{M}^1$ $\mathbf{M}^2$, respectively and the two complex-valued functions $f_1$ and $f_2$ :
  
$$
f_1(s)=\sum_{j=1}^m \frac{v_j}{s+i \omega_j} \quad f_2(s)=\sum_{j=1}^m \frac{w_j}{s+i \omega_j}
$$

- Step 4: Find numerically the common real positive zeros $s_i$ of $f_1$ and $f_2$.

The number of solutions will define the number of elements in the generalized Maxwell model and the values of the characteristic times are:

$$
\tau_i=\frac{1}{S_i}
$$

- Step 5: Compute the values of the moduli, by using the HW inverse identification described before.

#### Application to artificial data

As stated in the presentation of the $\mathrm{KN}+\mathrm{HW}$ method, the key point of the identification algorithm is the positive definition of the two matrices $\mathbf{M}^1$ and $\mathbf{M}^2$ and the necessary correction of the data.



**Table 1**
Generalized Maxwell model used to generate artificial data.

| $\tau_i(\mathrm{s})$ | 0.05 | 0.2 | 2   | 10  |
|-----------------------|------|-----|-----|-----|
| $E_i(\mathrm{Pa})$   | 400  | 300 | 150 | 8   |


#### Reference 

[1] Jalocha, D., A. Constantinescu, and R. Neviere. ‘Revisiting the Identification of Generalized Maxwell Models from Experimental Results’. International Journal of Solids and Structures 67–68 (August 2015): 169–81. https://doi.org/10.1016/j.ijsolstr.2015.04.018.

[2]


#### Appendix

*KN + HW* method

The Nevanlinna Pick interpolation problem consists in finding a complex function $F$ : $\mathbb{C} \longrightarrow \mathbb{C}$ interpolating the data pairs $\left(z_k, c_k\right)_{k=1, m}$, 

Moreover, a function $F(z)$ will be denoted as a $S$-function if:

$$
F(z)=\gamma+\int_0^{\infty} \frac{d \sigma(t)}{t-z}
$$

The viscoelastic identification problem can be transformed in the structure defined by the Pick-Nevalinna interpolation problem in the class of S-function. Actually the complex modulus of the viscoelastic problem is the conformal transform of an S-function:

$$
E^*(\omega)=E_0+\int_0^{\infty} H(\tau) \frac{i \omega \tau}{1+i \omega \tau} d \tau
$$
